In [299]:
import pandas as pd

df_operations = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Operations')

In [300]:
df_operations = df_operations[['event', 'Category']]
mask = df_operations['Category'].notna()
df_operations_u = df_operations[mask].reset_index(drop=True)
df_operations_u

,event,Category
0,LG_PHASE_1,Planned_downtime
1,LG_PHASE_2,Planned_downtime
2,LG_PHASE_3,Planned_downtime
3,LG_PHASE_4_1,Planned_downtime
4,LG_PHASE_5_2,Planned_downtime
5,RP_BROKEN_CABLE,Unplanned_downtime_losses
6,RP_BROKEN_ELECTRICAL,Unplanned_downtime_losses
7,RP_BROKEN_GENERATOR,Unplanned_downtime_losses
8,RP_BROKEN_HYDRAULIC,Unplanned_downtime_losses
9,RP_BROKEN_PT_1200,Unplanned_downtime_losses


In [301]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client 11.0', 'ODBC Driver 17 for SQL Server']


In [302]:
driver = 'ODBC Driver 17 for SQL Server'
server = 'BKZTKDSDB41,21433'
database = 'PIFD'

# Строка подключения
conn_str = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# SQL-запрос
query = """
SELECT 
    AFE.name COLLATE DATABASE_DEFAULT AS [equipment],
    AFET.name COLLATE DATABASE_DEFAULT AS [event],
    DATEADD(SECOND, (AVEvF.starttime / 10000000) - 63240134400 + 6 * 3600, '2005-01-01') AS starttime,
    IIF(AVEvF.endtime > 2638317188000000000, NULL, 
        DATEADD(SECOND, (AVEvF.endtime / 10000000) - 63240134400 + 6 * 3600, '2005-01-01')) AS endtime
FROM PIFD.dbo.AFEventFrame AS AVEvF
LEFT JOIN PIFD.dbo.AFElementTemplate AS AFET 
    ON AVEvF.fktemplateid = AFET.rid
LEFT JOIN PIFD.dbo.AFElement AS AFE 
    ON AVEvF.fkprimaryreferencedelement = AFE.rid
WHERE AFE.name COLLATE DATABASE_DEFAULT IN (
    SELECT DISTINCT equipment COLLATE DATABASE_DEFAULT 
    FROM PI_EXTRA.dbo.DR_REL_EQUIPMENT_EVENT
)
AND AFE.id IN (
    SELECT fkelementid 
    FROM PIFD.dbo.AFElementVersion 
    WHERE fktemplateid = (
        SELECT id 
        FROM PIFD.dbo.AFElementTemplate 
        WHERE name = 'RIG' 
        AND fkdatabaseid = (
            SELECT id FROM PIFD.dbo.AFDatabase WHERE name = 'DEM_DR'
        )
    )
)
"""

# Загрузка в DataFrame
try:
    with pyodbc.connect(conn_str) as conn:
        df_osi_operations = pd.read_sql(query, conn) # pyright: ignore[reportArgumentType]
        print(df_osi_operations.head())  # Проверка результата
except Exception as e:
    print("Ошибка при подключении или выполнении запроса:", e)

       equipment         event           starttime             endtime
0  DBKAZ40 BL №1     RP_REPAIR 2021-08-20 12:00:00 2021-09-02 17:00:00
1  DBKAZ40 BL №1  CV_LONG_STOP 2020-04-27 08:00:00 2020-08-15 08:00:00
2  DBKAZ40 BL №1        WT_PPR 2020-02-18 08:00:00 2020-02-18 09:00:00
3  DBKAZ40 BL №1      WT_LUNCH 2020-02-18 12:00:00 2020-02-18 13:00:00
4  DBKAZ40 BL №1        WT_PPR 2020-02-18 20:00:00 2020-02-18 21:00:00


In [303]:
df_osi_operations['duration'] = df_osi_operations['endtime'] - df_osi_operations['starttime']

df_osi_operations['duration'] = df_osi_operations['duration'].dt.total_seconds() / 3600

df_osi_operations


,equipment,event,starttime,endtime,duration
0,DBKAZ40 BL №1,RP_REPAIR,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0
1,DBKAZ40 BL №1,CV_LONG_STOP,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0
2,DBKAZ40 BL №1,WT_PPR,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0
3,DBKAZ40 BL №1,WT_LUNCH,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0
4,DBKAZ40 BL №1,WT_PPR,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0
...,...,...,...,...,...
557062,ЯЯЯЯ-ФЕЙК,DR_CORE_DRILLING,2024-09-29 21:00:00,2024-09-29 23:30:00,2.5
557063,ЯЯЯЯ-ФЕЙК,DR_REAMING,2024-09-30 11:30:00,2024-09-30 13:30:00,2.0
557064,ЯЯЯЯ-ФЕЙК,DR_PUMP_REPLACEMENT,2024-09-30 14:00:00,2024-09-30 15:30:00,1.5
557065,ЯЯЯЯ-ФЕЙК,DR_WELL_FLUSHING,2024-10-01 12:30:00,2024-10-01 15:00:00,2.5


In [304]:
df_rigs = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Rigs-Name')
df_rigs

,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,BurGeoProekt,PRAKLA_01-BGP,NaN,Direct
1,BurGeoProekt,PRAKLA_02-BGP,NaN,Direct
2,BurGeoProekt,PRAKLA_03-BGP,NaN,Direct
3,BurGeoProekt,PRAKLA_04-BGP,NaN,Direct
4,BurGeoProekt,ZIF1200_01-BGP,ZIF 1200 №1 Бургеопроект,Direct
...,...,...,...,...
157,VolkovGeology_#5,ZIF1200_36-E05,NaN,Direct
158,VolkovGeology_#5,ZIF1200_37-E05,NaN,Direct
159,VolkovGeology_#5,ZIF1200_38-E05,NaN,Direct
160,VolkovGeology_#5,ZIF1200_42-E05,NaN,Direct


In [305]:
df_circ_data = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Standart avarage meters')
df_circ_data

,Circ,"Standard avarage drilling, m/h","time to well drill, h"
0,Direct,2.7,210
1,RC,5.4,87


In [306]:
df_osi_operations['endtime'].max()

Timestamp('2025-06-03 09:00:00')

In [307]:
database_a = 'acqkzdem1'
# SQL-запрос
query_2 = """
SELECT 
    HOLEID,
    DrillCompany,
    DrillRig,
    HolePurpose,
    HoleStatus,
    ENDDATE,
    [DEPTH],
    [InvoiceDepth]
FROM [acqkzdem1].[dbo].[AV_AREVA_V_DRILLING_SPREADSHEET_old]
"""

# Строка подключения
conn_str = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database_a};Trusted_Connection=yes;'

# Загрузка данных в DataFrame
try:
    with pyodbc.connect(conn_str) as conn:
        df_meters = pd.read_sql(query_2, conn) # pyright: ignore[reportArgumentType]
        print(df_meters.head())  # Проверка
except Exception as e:
    print("Ошибка при подключении или выполнении запроса:", e)

           HOLEID  DrillCompany        DrillRig HolePurpose      HoleStatus  \
0  MSC_V____01_c_          None            None       WATER        ACCEPTED   
1      MCQ_N_01_1          None            None       PIEZO         PROJECT   
2    STC_V_2118_1  BurGeoProekt  ZIF1200_12-BGP       WATER  NOT PROFITABLE   
3    STC_V_2105_3  BurGeoProekt  ZIF1200_13-BGP       WATER        ACCEPTED   
4    STC_V_2101_2  BurGeoProekt  ZIF1200_13-BGP       WATER        ACCEPTED   

      ENDDATE  DEPTH InvoiceDepth  
0        None  600.0          600  
1        None    NaN         None  
2  2022-08-23  620.0          610  
3  2022-08-23  609.0          609  
4  2022-09-04  604.0          600  


In [308]:
df_meters

,HOLEID,DrillCompany,DrillRig,HolePurpose,HoleStatus,ENDDATE,DEPTH,InvoiceDepth
0,MSC_V____01_c_,None,None,WATER,ACCEPTED,None,600.0,600
1,MCQ_N_01_1,None,None,PIEZO,PROJECT,None,NaN,None
2,STC_V_2118_1,BurGeoProekt,ZIF1200_12-BGP,WATER,NOT PROFITABLE,2022-08-23,620.0,610
3,STC_V_2105_3,BurGeoProekt,ZIF1200_13-BGP,WATER,ACCEPTED,2022-08-23,609.0,609
4,STC_V_2101_2,BurGeoProekt,ZIF1200_13-BGP,WATER,ACCEPTED,2022-09-04,604.0,600
...,...,...,...,...,...,...,...,...
20116,TSU00_07_02___,VolkovGeology_#7,None,INJECT,ACCEPTED,2000-01-01,311.0,311
20117,TSU00_07_05___,VolkovGeology_#7,None,INJECT,ACCEPTED,2000-01-01,310.0,310
20118,TSU00_06_01_2_,VolkovGeology_#7,None,PRODUCT,ACCEPTED,2000-01-01,310.0,310
20119,TSU00_06_01___,VolkovGeology_#7,None,PRODUCT,LIQUID,2000-01-01,302.0,302


df_operations     - Название операций <br>
df_operations_u   - Название операций уникальные значения <br>
df_osi_operations - Время операций <br>
df_rigs           - Название агрегатов для связи <br>
df_circ_data      - Стандартные коэфициенты в зависим от циркуляции (скорость бурения в час, и время на одну скважину - час) <br>
df_meters         - Пробуренные метры

## Идея 
1 связать df_osi_operations - df_rigs (Взять за основу Rig-AcQuire не пустые)
2 связать df_osi_operations - df_df_operations_u


In [309]:
mask = df_rigs['Rig-AcQuire'].notna()
df_rigs = df_rigs[mask].reset_index(drop=True)
df_rigs.info()
df_osi_operations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   DrillCompany         162 non-null    object
 1   Rig-AcQuire          162 non-null    object
 2   Rig-osiDEM           74 non-null     object
 3   Tipe of circulation  162 non-null    object
dtypes: object(4)
memory usage: 5.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557067 entries, 0 to 557066
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   equipment  557067 non-null  object        
 1   event      557067 non-null  object        
 2   starttime  557067 non-null  datetime64[ns]
 3   endtime    557059 non-null  datetime64[ns]
 4   duration   557059 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(2)
memory usage: 21.3+ MB


In [310]:
df_osi_operations['equipment'] = df_osi_operations['equipment'].str.strip().str.lower()
df_rigs['Rig-osiDEM'] = df_rigs['Rig-osiDEM'].str.strip().str.lower()

df_merged = df_osi_operations.merge(
    df_rigs,
    left_on='equipment',
    right_on='Rig-osiDEM',
    how='inner'  # или 'inner', если нужны только совпадающие строки
)

df_merged

,equipment,event,starttime,endtime,duration,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,dbkaz40 bl №1,RP_REPAIR,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
1,dbkaz40 bl №1,CV_LONG_STOP,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
2,dbkaz40 bl №1,WT_PPR,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
3,dbkaz40 bl №1,WT_LUNCH,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
4,dbkaz40 bl №1,WT_PPR,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
...,...,...,...,...,...,...,...,...,...
505638,пбу змо-1500пс №6,WT_WAITING_FOR_LOGGING_5_2,2025-06-01 16:00:00,2025-06-01 19:00:00,3.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct
505639,пбу змо-1500пс №6,WT_WAITING_FOR_LOGGING_RESULT_5_2,2025-06-01 19:00:00,2025-06-01 21:00:00,2.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct
505640,пбу змо-1500пс №6,AS_MAST_DOWN,2025-06-01 22:00:00,2025-06-02 01:00:00,3.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct
505641,пбу змо-1500пс №6,AS_MAST_DOWN,2025-06-02 02:00:00,2025-06-02 09:00:00,7.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct


In [311]:
df_temp = df_merged.drop(columns=['equipment', 'Rig-osiDEM'])

In [312]:
df_operations_u

,event,Category
0,LG_PHASE_1,Planned_downtime
1,LG_PHASE_2,Planned_downtime
2,LG_PHASE_3,Planned_downtime
3,LG_PHASE_4_1,Planned_downtime
4,LG_PHASE_5_2,Planned_downtime
5,RP_BROKEN_CABLE,Unplanned_downtime_losses
6,RP_BROKEN_ELECTRICAL,Unplanned_downtime_losses
7,RP_BROKEN_GENERATOR,Unplanned_downtime_losses
8,RP_BROKEN_HYDRAULIC,Unplanned_downtime_losses
9,RP_BROKEN_PT_1200,Unplanned_downtime_losses


In [313]:
df_temp['event'] = df_temp['event'].str.strip().str.lower()
df_operations_u['event'] = df_operations_u['event'].str.strip().str.lower()

df_operations_total = df_temp.merge(df_operations_u, on='event', how='left')

df_operations_total['Category'] = df_operations_total['Category'].fillna('Speed_losess')

In [314]:
df_operations_total

,event,starttime,endtime,duration,DrillCompany,Rig-AcQuire,Tipe of circulation,Category
0,rp_repair,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0,KATCO,DBKAZ40-1,RC,Unplanned_downtime_losses
1,cv_long_stop,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0,KATCO,DBKAZ40-1,RC,Speed_losess
2,wt_ppr,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0,KATCO,DBKAZ40-1,RC,Planned_downtime
3,wt_lunch,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0,KATCO,DBKAZ40-1,RC,Planned_downtime
4,wt_ppr,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0,KATCO,DBKAZ40-1,RC,Planned_downtime
...,...,...,...,...,...,...,...,...
505638,wt_waiting_for_logging_5_2,2025-06-01 16:00:00,2025-06-01 19:00:00,3.0,KATCO,ZMO1500-6-KAT,Direct,Unplanned_downtime_losses
505639,wt_waiting_for_logging_result_5_2,2025-06-01 19:00:00,2025-06-01 21:00:00,2.0,KATCO,ZMO1500-6-KAT,Direct,Unplanned_downtime_losses
505640,as_mast_down,2025-06-01 22:00:00,2025-06-02 01:00:00,3.0,KATCO,ZMO1500-6-KAT,Direct,Speed_losess
505641,as_mast_down,2025-06-02 02:00:00,2025-06-02 09:00:00,7.0,KATCO,ZMO1500-6-KAT,Direct,Speed_losess


In [315]:
# Убедимся, что endtime — datetime (хотя уже подтверждено)
df_operations_total['endtime'] = pd.to_datetime(df_operations_total['endtime'])
df_operations_total = df_operations_total.dropna(subset=['endtime'])

# Добавим год и месяц
df_operations_total['year'] = df_operations_total['endtime'].dt.year.astype('int16')
df_operations_total['month'] = df_operations_total['endtime'].dt.month.astype('int8')

pivot_df = df_operations_total.pivot_table(
    index=['DrillCompany', 'Rig-AcQuire', 'year', 'month', 'Tipe of circulation'],
    columns='Category',
    values='duration',
    aggfunc='sum',
    fill_value=0  # если нужно подставить 0 вместо NaN
).reset_index()

pivot_df = pivot_df.sort_values(
    by=['DrillCompany', 'Rig-AcQuire', 'year', 'month'],
    ascending=[True, True, True, True]
).reset_index(drop=True)


pivot_df


Category,DrillCompany,Rig-AcQuire,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses
0,BurGeoProekt,ZIF1200_01-BGP,2020,2,Direct,0.0,0.0,276.0
1,BurGeoProekt,ZIF1200_01-BGP,2020,3,Direct,47.0,420.0,278.0
2,BurGeoProekt,ZIF1200_01-BGP,2020,4,Direct,17.0,182.0,28.0
3,BurGeoProekt,ZIF1200_01-BGP,2021,1,Direct,9.0,7015.0,93.0
4,BurGeoProekt,ZIF1200_01-BGP,2021,2,Direct,84.0,397.0,191.0
...,...,...,...,...,...,...,...,...
2805,TechnoService-Eng,ZIF1200_16-TSE,2023,3,Direct,89.0,509.0,90.0
2806,TechnoService-Eng,ZIF1200_16-TSE,2023,4,Direct,76.0,468.0,106.0
2807,TechnoService-Eng,ZIF1200_16-TSE,2023,5,Direct,76.0,486.0,119.0
2808,TechnoService-Eng,ZIF1200_16-TSE,2023,6,Direct,77.0,405.0,133.0


In [316]:
import calendar

def hours_in_month(row):
    days = calendar.monthrange(row['year'], row['month'])[1]
    return days * 24

pivot_df['h_in_month'] = pivot_df.apply(hours_in_month, axis=1)

pivot_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2810 entries, 0 to 2809
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DrillCompany               2810 non-null   object 
 1   Rig-AcQuire                2810 non-null   object 
 2   year                       2810 non-null   int16  
 3   month                      2810 non-null   int8   
 4   Tipe of circulation        2810 non-null   object 
 5   Planned_downtime           2810 non-null   float64
 6   Speed_losess               2810 non-null   float64
 7   Unplanned_downtime_losses  2810 non-null   float64
 8   h_in_month                 2810 non-null   int64  
dtypes: float64(3), int16(1), int64(1), int8(1), object(3)
memory usage: 162.0+ KB


In [323]:
# Преобразование ENDDATE в datetime формат
df_meters['ENDDATE'] = pd.to_datetime(df_meters['ENDDATE'], errors='coerce')

# Удаление строк с некорректными датами (NaT после преобразования)
df_meters.dropna(subset=['ENDDATE'], inplace=True)

df_meters.info()


<class 'pandas.core.frame.DataFrame'>
Index: 19435 entries, 2 to 20120
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   HOLEID        19435 non-null  object        
 1   DrillCompany  19434 non-null  object        
 2   DrillRig      19292 non-null  object        
 3   HolePurpose   19435 non-null  object        
 4   HoleStatus    19435 non-null  object        
 5   ENDDATE       19435 non-null  datetime64[ns]
 6   DEPTH         19356 non-null  float64       
 7   InvoiceDepth  19377 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 1.3+ MB


In [325]:
df_meters['year'] = df_meters['ENDDATE'].dt.year.astype('int16')
df_meters['month'] = df_meters['ENDDATE'].dt.month.astype('int8')


df_meters['HoleStatus'].value_counts()

HoleStatus
ACCEPTED           17652
LIQUID              1417
REFUSED              179
NOT PROFITABLE       151
PAID_MudLoss          17
REFUSED_MudLoss       10
ECHEC_MudLoss          4
ECHEC                  4
PROJECT                1
Name: count, dtype: int64

In [ ]:
import numpy as np

df_meters = df_meters.dropna(subset=['year', 'month', 'DEPTH'])

pivot_meters = df_meters.pivot_table(
    index=['DrillRig', 'year', 'month'],  # группируем по году, месяцу и статусу
    columns='HoleStatus',
    values='DEPTH',
    aggfunc='sum',
    fill_value=0  # если нужно заполнить NaN нулями
).reset_index()

pivot_meters = pivot_meters.sort_values(
    by=['DrillRig', 'year', 'month'],
    ascending=[True, True, True]
).reset_index(drop=True)

meters = pivot_meters

# 1. Добавляем TOTAL_DRILLED
pivot_meters['TOTAL_DRILLED'] = (
    pivot_meters['ACCEPTED'] +
    pivot_meters['LIQUID'] +
    pivot_meters['NOT PROFITABLE']
)

# 2. Рассчитываем коэффициент (ACCEPTED + NOT PROFITABLE) / TOTAL_DRILLED
pivot_meters['ACCEPTED_NOTPROF_RATIO'] = (
    (pivot_meters['ACCEPTED'] + pivot_meters['NOT PROFITABLE']) /
    pivot_meters['TOTAL_DRILLED']
).replace([np.inf, -np.inf], np.nan).fillna(0)

# 3. Оставляем только нужные колонки
pivot_meters = pivot_meters[[
    'DrillRig', 'year', 'month', 'TOTAL_DRILLED', 'ACCEPTED_NOTPROF_RATIO'
]]


pivot_meters


HoleStatus,DrillRig,year,month,TOTAL_DRILLED,ACCEPTED_NOTPROF_RATIO
0,DBKAZ40-1,2007,11,486.20,1.000000
1,DBKAZ40-1,2007,12,949.60,0.503370
2,DBKAZ40_1-KAT,2007,7,480.20,0.000000
3,DBKAZ40_1-KAT,2007,8,1935.60,0.754081
4,DBKAZ40_1-KAT,2007,9,1438.00,1.000000
...,...,...,...,...,...
5888,ZMO1500-6-KAT,2025,1,1677.00,1.000000
5889,ZMO1500-6-KAT,2025,2,1279.00,1.000000
5890,ZMO1500-6-KAT,2025,3,1686.25,1.000000
5891,ZMO1500-6-KAT,2025,4,1275.00,1.000000


In [ ]:
pivot_df['Rig-AcQuire'] = pivot_df['Rig-AcQuire'].str.strip().str.upper()
pivot_meters['DrillRig'] = pivot_meters['DrillRig'].str.strip().str.upper()


total_merged_df = pd.merge(
    pivot_df,
    pivot_meters,
    left_on=['Rig-AcQuire', 'year', 'month'],
    right_on=['DrillRig', 'year', 'month'],
    how='left'  
)

total_merged_df.drop(columns='Rig-AcQuire', inplace=True)

total_merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2810 entries, 0 to 2809
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DrillCompany               2810 non-null   object 
 1   year                       2810 non-null   int16  
 2   month                      2810 non-null   int8   
 3   Tipe of circulation        2810 non-null   object 
 4   Planned_downtime           2810 non-null   float64
 5   Speed_losess               2810 non-null   float64
 6   Unplanned_downtime_losses  2810 non-null   float64
 7   h_in_month                 2810 non-null   int64  
 8   DrillRig                   2412 non-null   object 
 9   TOTAL_DRILLED              2412 non-null   float64
 10  ACCEPTED_NOTPROF_RATIO     2412 non-null   float64
dtypes: float64(5), int16(1), int64(1), int8(1), object(3)
memory usage: 205.9+ KB


In [ ]:
total_merged_df['Planned_production_time'] = total_merged_df['h_in_month'] - total_merged_df['Planned_downtime']

total_merged_df['Planned_factor'] = total_merged_df['Planned_production_time'] / total_merged_df['h_in_month']

# 1. Planned Production Time (PPT) с ограничением снизу
total_merged_df['Planned_production_time'] = (
    total_merged_df['h_in_month'] - total_merged_df['Planned_downtime']
).clip(lower=0)

# 2. Planned Factor (в долях или в %)
total_merged_df['Planned_factor'] = (
    total_merged_df['Planned_production_time'] / total_merged_df['h_in_month']
).clip(lower=0)


total_merged_df.describe(include="all")


,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,TOTAL_DRILLED,ACCEPTED_NOTPROF_RATIO,Planned_production_time,Planned_factor
count,2810,2810.000000,2810.000000,2810,2810.000000,2810.000000,2810.000000,2810.000000,2412,2412.000000,2412.000000,2810.000000,2810.000000
unique,5,NaN,NaN,2,NaN,NaN,NaN,NaN,66,NaN,NaN,NaN,NaN
top,BurGeoProekt,NaN,NaN,Direct,NaN,NaN,NaN,NaN,ZIF1200_29-BGP,NaN,NaN,NaN,NaN
freq,1057,NaN,NaN,2754,NaN,NaN,NaN,NaN,61,NaN,NaN,NaN,NaN
mean,NaN,2022.602847,6.271886,NaN,81.566548,468.630783,142.134698,729.659786,NaN,1204.038951,0.932183,648.093238,0.888239
std,NaN,1.452313,3.505185,NaN,31.561002,247.711478,179.684916,20.158740,NaN,527.493165,0.179872,35.797836,0.043138
min,NaN,2020.000000,1.000000,NaN,0.000000,0.000000,0.000000,672.000000,NaN,80.000000,0.000000,379.000000,0.509409
25%,NaN,2021.000000,3.000000,NaN,69.000000,426.000000,72.000000,720.000000,NaN,869.000000,1.000000,630.000000,0.866667
50%,NaN,2023.000000,6.000000,NaN,85.000000,496.000000,106.000000,744.000000,NaN,1090.500000,1.000000,648.000000,0.883929
75%,NaN,2024.000000,9.000000,NaN,97.000000,541.000000,156.000000,744.000000,NaN,1537.000000,1.000000,667.000000,0.905556


In [ ]:
# Gross Operating Time (GOT)
total_merged_df['Gross_operating_time'] = total_merged_df['Planned_production_time'] - total_merged_df['Unplanned_downtime_losses']

# Availability (доступность)
total_merged_df['Availability'] = total_merged_df['Gross_operating_time'] / total_merged_df['h_in_month']

# Gross Operating Time (GOT) с ограничением снизу
total_merged_df['Gross_operating_time'] = (
    total_merged_df['Planned_production_time'] - total_merged_df['Unplanned_downtime_losses']
).clip(lower=0)

# Availability (доступность) с ограничением снизу
total_merged_df['Availability'] = (
    total_merged_df['Gross_operating_time'] / total_merged_df['h_in_month']
).clip(lower=0)


total_merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2810 entries, 0 to 2809
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DrillCompany               2810 non-null   object 
 1   year                       2810 non-null   int16  
 2   month                      2810 non-null   int8   
 3   Tipe of circulation        2810 non-null   object 
 4   Planned_downtime           2810 non-null   float64
 5   Speed_losess               2810 non-null   float64
 6   Unplanned_downtime_losses  2810 non-null   float64
 7   h_in_month                 2810 non-null   int64  
 8   DrillRig                   2412 non-null   object 
 9   TOTAL_DRILLED              2412 non-null   float64
 10  ACCEPTED_NOTPROF_RATIO     2412 non-null   float64
 11  Planned_production_time    2810 non-null   float64
 12  Planned_factor             2810 non-null   float64
 13  Gross_operating_time       2810 non-null   float

In [ ]:
# Назначаем коэффициенты по типу циркуляции
circulation_coeffs = df_circ_data.set_index('Circ')['Standard avarage drilling, m/h'].to_dict()

# Присваиваем коэффициенты каждому ряду
total_merged_df['circulation_coeff'] = total_merged_df['Tipe of circulation'].map(circulation_coeffs)

# Вычисляем потенциальную глубину бурения на основе Gross Operating Time
total_merged_df['Potential_depth'] = (
    total_merged_df['Gross_operating_time'] * total_merged_df['circulation_coeff']
).round(1)



total_merged_df


,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,TOTAL_DRILLED,ACCEPTED_NOTPROF_RATIO,Planned_production_time,Planned_factor,Gross_operating_time,Availability,circulation_coeff,Potential_depth
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,80.0,0.000000,696.0,1.000000,420.0,0.603448,2.7,1134.0
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,704.0,1.000000,697.0,0.936828,419.0,0.563172,2.7,1131.3
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,697.0,1.000000,703.0,0.976389,675.0,0.937500,2.7,1822.5
3,BurGeoProekt,2021,1,Direct,9.0,7015.0,93.0,744,NaN,NaN,NaN,735.0,0.987903,642.0,0.862903,2.7,1733.4
4,BurGeoProekt,2021,2,Direct,84.0,397.0,191.0,672,ZIF1200_01-BGP,1001.0,1.000000,588.0,0.875000,397.0,0.590774,2.7,1071.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2805,TechnoService-Eng,2023,3,Direct,89.0,509.0,90.0,744,ZIF1200_16-TSE,598.0,1.000000,655.0,0.880376,565.0,0.759409,2.7,1525.5
2806,TechnoService-Eng,2023,4,Direct,76.0,468.0,106.0,720,ZIF1200_16-TSE,1150.0,1.000000,644.0,0.894444,538.0,0.747222,2.7,1452.6
2807,TechnoService-Eng,2023,5,Direct,76.0,486.0,119.0,744,ZIF1200_16-TSE,574.0,1.000000,668.0,0.897849,549.0,0.737903,2.7,1482.3
2808,TechnoService-Eng,2023,6,Direct,77.0,405.0,133.0,720,ZIF1200_16-TSE,1063.0,1.000000,643.0,0.893056,510.0,0.708333,2.7,1377.0


In [ ]:
def calculate_net_operating_time(row):
    try:
        drilled = row['TOTAL_DRILLED']
        coeff = row['circulation_coeff']
        if pd.notnull(drilled) and pd.notnull(coeff) and coeff > 0:
            result = drilled / coeff
            return max(result, 0)  # отрицательные значения → 0
        else:
            return 0
    except:
        return 0

total_merged_df['Net_operating_time'] = total_merged_df.apply(calculate_net_operating_time, axis=1).round(1)


In [ ]:
total_merged_df.head(2)

,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,TOTAL_DRILLED,ACCEPTED_NOTPROF_RATIO,Planned_production_time,Planned_factor,Gross_operating_time,Availability,circulation_coeff,Potential_depth,Net_operating_time
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,80.0,0.0,696.0,1.000000,420.0,0.603448,2.7,1134.0,29.6
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,704.0,1.0,697.0,0.936828,419.0,0.563172,2.7,1131.3,260.7


In [ ]:
def calculate_performance(row):
    try:
        net_time = row['Net_operating_time']
        gross_time = row['Gross_operating_time']
        if pd.notnull(net_time) and pd.notnull(gross_time) and gross_time > 0:
            result = net_time / gross_time
            return max(result, 0)  # убрать отрицательные
        else:
            return 0
    except:
        return 0

total_merged_df['Performance'] = total_merged_df.apply(calculate_performance, axis=1)


total_merged_df.head(3)

,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,TOTAL_DRILLED,ACCEPTED_NOTPROF_RATIO,Planned_production_time,Planned_factor,Gross_operating_time,Availability,circulation_coeff,Potential_depth,Net_operating_time,Performance
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,80.0,0.0,696.0,1.000000,420.0,0.603448,2.7,1134.0,29.6,0.070476
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,704.0,1.0,697.0,0.936828,419.0,0.563172,2.7,1131.3,260.7,0.622196
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,697.0,1.0,703.0,0.976389,675.0,0.937500,2.7,1822.5,258.1,0.382370


In [ ]:
total_merged_df = total_merged_df.rename(columns={'ACCEPTED_NOTPROF_RATIO': 'quality'})

total_merged_df['OEE'] = (
    total_merged_df['Availability'] *
    total_merged_df['Performance'] *
    total_merged_df['quality']
)


In [ ]:
total_merged_df.head(3)

,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,TOTAL_DRILLED,quality,Planned_production_time,Planned_factor,Gross_operating_time,Availability,circulation_coeff,Potential_depth,Net_operating_time,Performance,OEE
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,80.0,0.0,696.0,1.000000,420.0,0.603448,2.7,1134.0,29.6,0.070476,0.000000
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,704.0,1.0,697.0,0.936828,419.0,0.563172,2.7,1131.3,260.7,0.622196,0.350403
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,697.0,1.0,703.0,0.976389,675.0,0.937500,2.7,1822.5,258.1,0.382370,0.358472


In [ ]:
total_merged_df['TRS'] = (
    total_merged_df['OEE'] * total_merged_df['Planned_factor']
)

total_merged_df.head(3)


,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,TOTAL_DRILLED,...,Planned_production_time,Planned_factor,Gross_operating_time,Availability,circulation_coeff,Potential_depth,Net_operating_time,Performance,OEE,TRS
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,80.0,...,696.0,1.000000,420.0,0.603448,2.7,1134.0,29.6,0.070476,0.000000,0.000000
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,704.0,...,697.0,0.936828,419.0,0.563172,2.7,1131.3,260.7,0.622196,0.350403,0.328268
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,697.0,...,703.0,0.976389,675.0,0.937500,2.7,1822.5,258.1,0.382370,0.358472,0.350008


In [ ]:
total_merged_df.columns

Index(['DrillCompany', 'year', 'month', 'Tipe of circulation',
       'Planned_downtime', 'Speed_losess', 'Unplanned_downtime_losses',
       'h_in_month', 'DrillRig', 'TOTAL_DRILLED', 'quality',
       'Planned_production_time', 'Planned_factor', 'Gross_operating_time',
       'Availability', 'circulation_coeff', 'Potential_depth',
       'Net_operating_time', 'Performance', 'OEE', 'TRS'],
      dtype='object')

In [ ]:
total_merged_df.columns = [
    "Drill Company",
    "Year",
    "Month",
    "Type Of Circulation",
    "Planned Downtime",
    "Speed Losses",
    "Unplanned Downtime Losses",
    "H In Month",
    "Drill Rig",
    "Total Drilled",
    "Quality",
    "Planned Production Time",
    "Planned Factor",
    "Gross Operating Time",
    "Availability",
    "Circulation Coeff",
    "Potential Depth",
    "Net Operating Time",
    "Performance",
    "OEE",
    "TRS"
]


total_merged_df.describe(include='all')


,Drill Company,Year,Month,Type Of Circulation,Planned Downtime,Speed Losses,Unplanned Downtime Losses,H In Month,Drill Rig,Total Drilled,...,Planned Production Time,Planned Factor,Gross Operating Time,Availability,Circulation Coeff,Potential Depth,Net Operating Time,Performance,OEE,TRS
count,2810,2810.000000,2810.000000,2810,2810.000000,2810.000000,2810.000000,2810.000000,2412,2412.000000,...,2810.000000,2810.000000,2810.000000,2810.000000,2810.000000,2810.000000,2810.000000,2810.000000,2412.000000,2412.000000
unique,5,NaN,NaN,2,NaN,NaN,NaN,NaN,66,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,BurGeoProekt,NaN,NaN,Direct,NaN,NaN,NaN,NaN,ZIF1200_29-BGP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1057,NaN,NaN,2754,NaN,NaN,NaN,NaN,61,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2022.602847,6.271886,NaN,81.566548,468.630783,142.134698,729.659786,NaN,1204.038951,...,648.093238,0.888239,513.966370,0.704524,2.753808,1408.472633,379.204342,0.741262,0.565146,0.493838
std,NaN,1.452313,3.505185,NaN,31.561002,247.711478,179.684916,20.158740,NaN,527.493165,...,35.797836,0.043138,104.039574,0.142028,0.377408,315.525012,235.291132,0.487517,0.268346,0.227770
min,NaN,2020.000000,1.000000,NaN,0.000000,0.000000,0.000000,672.000000,NaN,80.000000,...,379.000000,0.509409,0.000000,0.000000,2.700000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,2021.000000,3.000000,NaN,69.000000,426.000000,72.000000,720.000000,NaN,869.000000,...,630.000000,0.866667,483.500000,0.663978,2.700000,1313.600000,194.100000,0.423629,0.370867,0.337022
50%,NaN,2023.000000,6.000000,NaN,85.000000,496.000000,106.000000,744.000000,NaN,1090.500000,...,648.000000,0.883929,534.000000,0.734459,2.700000,1447.200000,377.800000,0.745070,0.522083,0.465439
75%,NaN,2024.000000,9.000000,NaN,97.000000,541.000000,156.000000,744.000000,NaN,1537.000000,...,667.000000,0.905556,569.000000,0.775000,2.700000,1541.700000,560.400000,1.045821,0.761060,0.665164


In [ ]:
total_merged_df['Speed Losses Calculated'] = (
    total_merged_df['Gross Operating Time'] - total_merged_df['Net Operating Time']
).clip(lower=0)  # Чтобы отрицательных не было

In [ ]:
df_losses = pd.melt(
    total_merged_df,
    id_vars=[
        'Drill Company',
        'Drill Rig',
        'Year',
        'Month',
        'Type Of Circulation'
    ],
    value_vars=[
        'Planned Downtime',
        'Unplanned Downtime Losses',
        'Speed Losses Calculated',
        'Speed Losses'
    ],
    var_name='Loss Type',
    value_name='Loss Value'
)

df_losses.describe(include='all')

,Drill Company,Drill Rig,Year,Month,Type Of Circulation,Loss Type,Loss Value
count,11240,9648,11240.000000,11240.000000,11240,11240,11240.000000
unique,5,66,NaN,NaN,2,4,NaN
top,BurGeoProekt,ZIF1200_29-BGP,NaN,NaN,Direct,Planned Downtime,NaN
freq,4228,244,NaN,NaN,11016,2810,NaN
mean,NaN,NaN,2022.602847,6.271886,NaN,NaN,216.002936
std,NaN,NaN,1.452119,3.504717,NaN,NaN,235.187285
min,NaN,NaN,2020.000000,1.000000,NaN,NaN,0.000000
25%,NaN,NaN,2021.000000,3.000000,NaN,NaN,72.000000
50%,NaN,NaN,2023.000000,6.000000,NaN,NaN,115.000000
75%,NaN,NaN,2024.000000,9.000000,NaN,NaN,390.000000


In [ ]:
df_productivity = pd.melt(
    total_merged_df,
    id_vars=[
        'Drill Company',
        'Drill Rig',
        'Year',
        'Month',
        'Type Of Circulation'
    ],
    value_vars=[
        'Planned Factor',
        'Availability',
        'Performance',
        'Quality',
        'OEE',
        'TRS'
    ],
    var_name='Productivity Type',
    value_name='Productivity Value'
)

In [ ]:
df_operations_total['Category'].value_counts()

Category
Speed_losess                 247936
Planned_downtime             183456
Unplanned_downtime_losses     74243
Name: count, dtype: int64

In [ ]:
df_losses['Loss Type'].value_counts()

Loss Type
Planned Downtime             2810
Unplanned Downtime Losses    2810
Speed Losses Calculated      2810
Speed Losses                 2810
Name: count, dtype: int64

In [ ]:
df_operations_total['Category'] = df_operations_total['Category'].replace({
    'Speed_losess': 'Speed Losses',
    'Planned_downtime': 'Planned Downtime',
    'Unplanned_downtime_losses': 'Unplanned Downtime Losses'
})

df_operations_total['Category'].value_counts()

Category
Speed Losses                 247936
Planned Downtime             183456
Unplanned Downtime Losses     74243
Name: count, dtype: int64

In [ ]:
df_operations_total.columns = [
    "Event",
    "Start Time",
    "End Time",
    "Duration",
    "Drill Company",
    "Rig",
    "Type Of Circulation",
    "Category",
    "Year",
    "Month"
]


In [ ]:
df_operations_total['Event Category'] = df_operations_total['Event'].str.split('_').str[0].str.upper()
df_operations_total['Event'] = df_operations_total['Event'].str.split('_', n=1).str[1].str.capitalize()

df_events_duration = df_operations_total[
    [
        "Year",
        "Month",
        "Category",
        "Drill Company",
        "Rig",
        "Type Of Circulation",
        "Event Category",
        "Event",
        "Duration"
    ]
]
df_events_duration['Event'].value_counts()

Event
Ppr                   136470
Pilot_drilling         52792
Reaming                28792
Development            23623
Drill_rod_handling     19908
                       ...  
Swabb_locked               7
Air_lift                   5
Screen_destroy             5
Mud_pit_filling            4
Mantle_compressor          1
Name: count, Length: 107, dtype: int64

In [ ]:
df_events_duration.sort_values(
    by=['Year', 'Month', 'Rig', 'Category', 'Duration'],
    ascending=[False, True, True, True, False],
    inplace=True
)

df_events_duration

,Year,Month,Category,Drill Company,Rig,Type Of Circulation,Event Category,Event,Duration
6591,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Ppr,1.0
6592,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Lunch,1.0
6594,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Ppr,1.0
6595,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Ppr,1.0
6597,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Lunch,1.0
...,...,...,...,...,...,...,...,...,...
279250,2020,12,Unplanned Downtime Losses,BurGeoProekt,ZIF1200_29-BGP,Direct,WT,Waiting_for_logging_result,1.0
279269,2020,12,Unplanned Downtime Losses,BurGeoProekt,ZIF1200_29-BGP,Direct,WT,Waiting_for_logging_result,1.0
279274,2020,12,Unplanned Downtime Losses,BurGeoProekt,ZIF1200_29-BGP,Direct,WT,Waiting_for_logging_result,1.0
279279,2020,12,Unplanned Downtime Losses,BurGeoProekt,ZIF1200_29-BGP,Direct,WT,Waiting_for_logging,1.0


In [ ]:
meters

HoleStatus,DrillRig,year,month,ACCEPTED,CONSTRUCTION,LIQUID,NOT PROFITABLE,TOTAL_DRILLED,ACCEPTED_NOTPROF_RATIO
0,DBKAZ40-1,2007,11,486.20,0.0,0.0,0.0,486.20,1.000000
1,DBKAZ40-1,2007,12,478.00,0.0,471.6,0.0,949.60,0.503370
2,DBKAZ40_1-KAT,2007,7,0.00,0.0,480.2,0.0,480.20,0.000000
3,DBKAZ40_1-KAT,2007,8,1459.60,0.0,476.0,0.0,1935.60,0.754081
4,DBKAZ40_1-KAT,2007,9,1438.00,0.0,0.0,0.0,1438.00,1.000000
...,...,...,...,...,...,...,...,...,...
5888,ZMO1500-6-KAT,2025,1,1677.00,0.0,0.0,0.0,1677.00,1.000000
5889,ZMO1500-6-KAT,2025,2,1279.00,0.0,0.0,0.0,1279.00,1.000000
5890,ZMO1500-6-KAT,2025,3,1686.25,0.0,0.0,0.0,1686.25,1.000000
5891,ZMO1500-6-KAT,2025,4,1275.00,0.0,0.0,0.0,1275.00,1.000000
